# Machine Learning Project

__Pia CHANCEREL - Raphael LASRY - Maxime POLI__

Based on the article :

_A Continuation Method for Semi-Supervised SVMs_

Olivier Chapelle $\hspace{3.9cm}$ olivier.chapelle@tuebingen.mpg.de

Mingmin Chi $\hspace{4.5cm}$ mingmin.chi@tuebingen.mpg.de

Alexander Zien $\hspace{4.1cm}$ alexander.zien@tuebingen.mpg.de


Max Planck Institute for Biological Cybernetics, Tübingen, Germany

https://dl.acm.org/doi/pdf/10.1145/1143844.1143868?download=true

In [ ]:
import numpy as np
import scipy as sc
import pandas as pd
from tqdm import tqdm
import time as time
import matplotlib.pyplot as plt

# 20newsgroup

20newsgroup dataset: https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

Dataset of an old forum. We will only focus on messages related to windows and mac. The goal is to predict the subject of the message (windows or mac) thanks to a $S^3VM$ method.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures

# print(newsgroups_train.DESCR) # Documentation of how to use the dataset

In [ ]:
cat = ['comp.sys.mac.hardware', 'comp.os.ms-windows.misc']
# cat = ['sci.crypt', 'sci.electronics']
newsgroups_train = fetch_20newsgroups(subset = 'train', categories = cat) 
newsgroups_test = fetch_20newsgroups(subset = 'test', categories = cat) 

print('training set')
print('posts', newsgroups_train.filenames.shape) # Text, content of the messages
print('class', newsgroups_train.target.shape) # In which categories the message should be classified
print('test set')
print('posts', newsgroups_test.filenames.shape)
print('class', newsgroups_test.target.shape)

In [ ]:
# Converting text to vectors
vectorizer = TfidfVectorizer(encoding='latin1')
# Pas sûr de ma solution mais ça a l'air d'être ça
vectorizer.fit(newsgroups_train.data+newsgroups_test.data) # From a text message to a sparse matrix
vectors_train = vectorizer.transform(newsgroups_train.data)
vectors_test = vectorizer.transform(newsgroups_test.data)
print(vectors_train.shape)
print(vectors_test.shape)

In [ ]:
# Nombre moyen de features non nulles
vectors_test.nnz/float(vectors_test.shape[0])

In [ ]:
#print(vectors_train) # Sparse matrix. The first index is the index of the message and the second ones are the indexes whithin this matrix where the value isn't a 0.

# House Data

In [ ]:
# récupération des données
data = pd.read_csv('data.csv')
data['date_int'] = pd.to_datetime(data['date']).dt.strftime("%Y%m%d").astype(int)
city = pd.get_dummies(pd.Categorical(data['city']), prefix='city')
X_housedata = np.concatenate([data[['date_int', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']].to_numpy(), city.to_numpy()], axis=1)
Y_housedata = data['price'].to_numpy()

# séparation train, val, test
r = np.random.permutation(len(X_housedata))
X_train = X_housedata[r[0:3500], :]
Y_train = Y_housedata[r[0:3500]]
X_val = X_housedata[r[3500:4000], :]
Y_val = Y_housedata[r[3500:4000]]
X_test = X_housedata[r[4000:], :]
Y_test = Y_housedata[r[4000:]]

# standardisation
X_train = (X_train - np.mean(X_train, axis = 0)) / (np.std(X_train, axis = 0) + 10 ** (-15))
X_val = (X_val - np.mean(X_val, axis = 0)) / (np.std(X_val, axis = 0) + 10 ** (-15))
X_test = (X_test - np.mean(X_test, axis = 0)) / (np.std(X_test, axis = 0) + 10 ** (-15))

# séparation en 2 classes
Y_train = 2 * np.array(np.array(Y_train) - np.median(Y_train) > 0).astype(int) - 1
Y_val = 2 * np.array(np.array(Y_val) - np.median(Y_val) > 0).astype(int) - 1
Y_test = 2 * np.array(np.array(Y_test) - np.median(Y_test) > 0).astype(int) - 1

# S3VM

In [ ]:
# test de multiplication
X = np.ones((3, 4))
Y = np.arange(3)
print(X, Y)
print(np.multiply(X.T, Y).T)
print((X.T * Y).T)

In [475]:
import warnings
warnings.filterwarnings("ignore")

In [496]:
class S3VMClassifier():
    """
        S3VM class as defined in the article.
    """
    def __init__(self, C=1.0, C_=1.0, lr=0.01, s=3, sparse_data=False):
        self.C = C
        self.C_ = C_
        self.lr = lr
        self.s = s
        self.loss_list = []
        self.sparse_data = sparse_data
        self.X = None
        self.Y = None
        self.X_ = None
        self.w = None
        self.b = None
        
    def compute_gamma(self, epochs):
        """
            Compute the list of values of gamma (defined at the §3.2).
        """
        m, d = self.X_.shape

        if self.sparse_data:
            matrix = sc.sparse.csr_matrix((d, d))
            for i in range(m):
                matrix += self.X_[i].T@self.X_[i] / sc.sparse.linalg.norm(self.X_[i]) ** 3
            lamb_max = np.real(np.sort(sc.sparse.linalg.eigs(matrix, return_eigenvectors=False))[-1])
            gamma_0 = (self.C_ * lamb_max) ** (2 / 3) / (2 * self.s) ** (1 / 3)
            gamma_end = 1 / (epochs * 2 * self.s * sc.sparse.linalg.norm(self.X_, axis=1).max() ** 2)
            
        else:
            matrix = np.zeros((d, d))
            for i in range(m):
                matrix += self.X_[i].T@self.X_[i] / np.linalg.norm(self.X_[i]) ** 3
            lamb_max = np.sort(np.linalg.eigvals(matrix))[- 1]
            gamma_0 = (self.C_ * lamb_max) ** (2 / 3) / (2 * self.s) ** (1 / 3)
            gamma_end = 1 / (epochs * 2 * self.s * np.linalg.norm(self.X_, axis=1).max() ** 2)
            
        gamma_list = [(gamma_end / gamma_0) ** (i / epochs) * gamma_0 for i in range(epochs)]
        return gamma_list
    
    def loss(self, gamma):
        """
        !!! Pas corrigée pour sparse data
            Compute the convolved loss (defined at the end of the §3.1).
        """
        d = self.X.shape[1]
        if self.sparse_data:
            a = 1 + 2 * gamma * self.s * sc.sparse.linalg.norm(self.X_, axis = 1) ** 2
            print(self.X.shape, self.w.shape, self.X.dot(self.w).shape)
            e = (self.Y * (self.X.dot(self.w) + self.b) - 1) / np.sqrt(2 * gamma) / sc.sparse.linalg.norm(self.X, axis = 1)
            L_labelled = self.C * sc.sparse.csr_matrix.sum(gamma * sc.sparse.linalg.norm(self.X, axis = 1) / np.sqrt(2) 
                                         * (np.exp(- e ** 2) / np.sqrt(np.pi) - e * sc.special.erfc(e)))
            L_unlabelled = self.C_ * sc.sparse.csr_matrix.sum(1 / np.sqrt(a) * np.exp(- self.s * (self.X_.dot(self.w) + self.b) ** 2 / a))

        else:
            a = 1 + 2 * gamma * self.s * np.linalg.norm(self.X_, axis = 1) ** 2
            e = (self.Y * (self.X.dot(self.w) + self.b) - 1) / np.sqrt(2 * gamma) / np.linalg.norm(self.X, axis = 1)
            L_labelled = self.C * np.sum(gamma * np.linalg.norm(self.X, axis = 1) / np.sqrt(2) 
                                         * (np.exp(- e ** 2) / np.sqrt(np.pi) - e * sc.special.erfc(e)))
            L_unlabelled = self.C_ * np.sum(1 / np.sqrt(a) * np.exp(- self.s * (self.X_.dot(self.w) + self.b) ** 2 / a))

        return np.dot(self.w, self.w) / 2 + gamma * d / 2 + L_labelled + L_unlabelled
    
    def gradient_loss(self, gamma, labelled):
        """
            Compute the gradient of the loss (defined at the end of the §3.1).
        """
        if self.sparse_data:
            a = 1 + 2 * gamma * self.s * sc.sparse.linalg.norm(self.X_, axis = 1) ** 2
            e = (self.Y * (self.X.dot(self.w.T).todense() + self.b) - 1) / np.sqrt(2 * gamma) / sc.sparse.linalg.norm(self.X, axis = 1)
            e = e.T
            if labelled:
                yy = np.expand_dims(self.Y, axis=1)
                ww = np.multiply(sc.special.erfc(e), yy).T
                uu = sc.sparse.csr_matrix.multiply(self.X.T, ww)
                dL_labelled = self.C / 2 * np.sum(uu.T, axis=0)
                return self.w - dL_labelled
            else:
                xx = (self.X_.dot(self.w.T).todense() + self.b).T
                yy =  2 * self.s * xx / a ** (3 / 2)
                ww = np.multiply(yy, np.exp(- self.s * np.multiply(xx,xx) / a))
                uu = sc.sparse.csr_matrix.multiply(self.X_.T, ww)
                dL_unlabelled = self.C_ * np.sum(uu.T, axis=0)
                return self.w - dL_unlabelled
            
        else:
            a = 1 + 2 * gamma * self.s * np.linalg.norm(self.X_, axis = 1) ** 2
            e = (self.Y * (self.X.dot(self.w) + self.b) - 1) / np.sqrt(2 * gamma) / np.linalg.norm(self.X, axis = 1)
            if labelled:
                dL_labelled = self.C / 2 * np.sum(np.multiply(self.X.T, sc.special.erfc(e) * self.Y).T, axis=0)
                return self.w - dL_labelled
            else:
                dL_unlabelled = self.C_ * np.sum(np.multiply(self.X_.T, 2 * self.s * (self.X_.dot(self.w) + self.b) / a ** (3 / 2)
                                             * np.exp(- self.s * (self.X_.dot(self.w) + self.b) ** 2 / a)), axis=1)
                return self.w - dL_unlabelled
    
    def minimize(self, gamma, epochs):
        """
            Optimise the parameters.
        """
        n, d = self.X.shape
        m = self.X_.shape[0]
        indexes = np.arange(n + m) - m # positive indexes --> labeled & negative indexes --> unlabeled
        if self.sparse_data:
            gw = sc.sparse.csr_matrix((n + m, d))
            dw = sc.sparse.csr_matrix((1,d))
        else:
            gw = np.zeros((n + m, d)) # vector of most recent gradient in w
            dw = np.zeros(d) # sum of gw
        for ep in range(epochs):
            np.random.shuffle(indexes)
            for k in range(n + m):
                i = indexes[k]
                labelled = (i >= 0) # labelled or unlabelled
                if i < 0 : # if it's the index of an unlabelled data
                    i = - i + n - 1 # its index is in gw
                dw -= gw[i]
                gw[i] = self.gradient_loss(gamma, labelled)
                dw += gw[i]
                coeff = self.lr / min((k + 1) + ep * (n + m), n + m)
                self.w -=  coeff * dw
                print(k, end='\r')
    
    def fit(self, X, Y, X_, epochs=1, iter_gamma=10): # epochs pour la minimisation, iter_gamma pour le nombre de gamma
        """
            Train the model on X_labelled, Y_labelled, X_unlabelled datas.
        """       
        self.X = X
        self.Y = Y
        
#         # Centering unlabOuielled data
#         if self.sparse_data:
#             values = X_[X_.nonzero()]
#             values -= sc.sparse.spmatrix.mean(values, axis = 1)
#             self.X_ = values
#         else:
#             mean = np.mean(X_, axis = 0)
#             X_ -= mean
#             X -= mean
        
        self.X_ = X_
        gamma_list = self.compute_gamma(iter_gamma)
        
        # warm start
        if (self.w is None) and (self.b is None) :
            _, d = self.X.shape
            if self.sparse_data:
                self.w = sc.sparse.csr_matrix((1,d))
            else:
                self.w = np.zeros(d)
            self.b = np.mean(Y)
        
        # continuation method
        #for gamma in tqdm(gamma_list):
        print(gamma_list)
        for gamma in gamma_list:
            print('gamma:', gamma)
            self.minimize(gamma, epochs)
            if not self.sparse_data:
                self.loss_list.append(self.loss(gamma))
        
        return

    def predict(self, X):
        """
            Predict the value of Y (-1 or 1).
        """
        if self.sparse_data:
            return 2*(self.w.dot(X.T).todense()+self.b >= 0)-1
        return 2*(self.w.T.dot(X.T) + self.b >= 0)-1
    
    def accuracy(self, X, Y):
        """
            Evaluate the accuracy of the prediction.
        """
        return (self.predict(X) == Y).mean()

In [ ]:
# pour les figures

def make_meshgrid(points, h=0.2):
    x_min, x_max = points[:,0].min() - 1, points[:,0].max() + 1
    y_min, y_max = points[:,1].min() - 1, points[:,1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, model, X, Y, **params):
    xx, yy = make_meshgrid(X)
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, **params)
    ax.scatter(X[:, 0], X[:, 1], c=Y, s=20, cmap=plt.cm.Spectral)
    ax.set_axis_off()

# Résultats

## Breast cancer

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC

dataset = load_breast_cancer()
X = dataset.data
y = 2*(dataset.target) -1

r = np.random.permutation(len(X))

test_size = 150
Xtest = X[r[:test_size]]
ytest = y[r[:test_size]]
n = 100
Xl = X[r[test_size:test_size+n]]
yl = y[r[test_size:test_size+n]]

In [ ]:
m = 10000
Xu = X[r[test_size+n:n+m+test_size]]
yu = y[r[test_size+n:n+m+test_size]]
# Xl = X[r[test_size+m:]]
# yl = y[r[test_size+m:]]
print(Xu.shape, Xl.shape)

In [ ]:
svm = SVC()
svm.fit(Xl, yl)
print("sagc", np.mean(svm.predict(Xtest) == ytest))

s3vm = S3VMClassifier(lr=0.0001)
s3vm.fit(Xl, yl, Xu)
print("s3vm", s3vm.accuracy(Xtest, ytest))

### Ensemble jouet

In [ ]:
# paramètres
total_data = 2000
proportion = 0.75
n = int(proportion * total_data)
m = total_data - n
sigma = 0.2

# labels
y = 2 * np.random.randint(2, size=n) - 1
y_ = 2 * np.random.randint(2, size=m) - 1 # sert juste à construire x_

# observations
x = np.zeros((n, 2))
for i in range(n):
        x[i,:] = np.random.normal((y[i]+1)/2, sigma, 2)
x_ = np.zeros((m, 2))
for i in range(m):
        x_[i,:] = np.random.normal((y_[i]+1)/2, sigma, 2)

# classifieur
s3vm = S3VMClassifier()
s3vm.fit(x, y, x_)
print("accuracy sur l'ensemble jouet :", s3vm.accuracy(x, y))

# affichage
fig, ax = plt.subplots(figsize=(8,8))
plot_contours(ax, s3vm, x, y, alpha=0.8)
ax.scatter(x[:, 0], x[:, 1], c=y, s=10, cmap=plt.cm.Spectral)
ax.scatter(x_[:, 0], x_[:, 1], c='black', s=10, label='unlabelled')
ax.legend()
ax.set_title("s3vm")
plt.show()

# loss
plt.plot(np.arange(len(s3vm.loss_list)), np.log10(s3vm.loss_list))
plt.xlabel("log(loss)")
plt.ylabel("epoch")
plt.title("evolution de la loss sur le cas jouet")
plt.show()

In [ ]:
# très long à run (ne run que si nécessaire)

def classify_toy(proportion) :

    n = int(proportion * total_data)
    n = min(n, total_data - 2) # car il faut au moins 2 données non labelisées
    n = max(n, 2) # car il faut au moins 2 données labelisées
    m = total_data - n

    y = 2 * np.random.randint(2, size=n) - 1
    x = np.zeros((n, 2))
    for i in range(n):
            x[i,:] = np.random.normal((y[i]+1)/2, sigma, 2)
            
    y_ = 2 * np.random.randint(2, size=m) - 1
    x_ = np.zeros((m, 2))
    for i in range(m):
            x_[i,:] = np.random.normal((y_[i]+1)/2, sigma, 2)

    s3vm = S3VMClassifier()
    s3vm.fit(x, y, x_)
    return s3vm.accuracy(x, y)

total_data = 2000
sigma = 0.6
list_proportion = np.linspace(0, 1, 50)
list_accuracy = np.zeros(len(list_proportion))
for i in range(len(list_proportion)):
    print("proportion " + str(i+1) + "/" + str(len(list_proportion)))
    sum = classify_toy(list_proportion[i])
    sum += classify_toy(list_proportion[i])
    sum += classify_toy(list_proportion[i])
    list_accuracy[i] = sum/3

fig = plt.figure(figsize=(9, 5))
plt.plot(list_proportion, list_accuracy)
plt.ylabel("accuracy")
plt.xlabel("proportion of labelled")
plt.title("Evolution de l'accuracy en fonction de la proportion de données labelisées")
plt.show()

### House Data

In [ ]:
s3vm = S3VMClassifier(sparse_data=True)
train_labels = 2*newsgroups_train.target[:N] -1
test_labels = 2*newsgroups_test.target-1

N = vectors_train.shape[0]
proportion = 0.95
n = int(proportion*N)
m = 10

s3vm.fit(vectors_train[:n, :], 
         train_labels[:n], 
         vectors_train[n:n+m, :], iter_gamma=10)
# print(s3vm.accuracy(vectors_test, test_labels))

[0.6719245550369743, 0.4642729019618588, 0.32079394312985715, 0.22165574065155383, 0.15315522133751938, 0.10582411154339812, 0.07312021416018248, 0.050523133535955486, 0.034909457687037196, 0.024121034280974625]
gamma: 0.6719245550369743
gamma: 0.4642729019618588


In [ ]:
svm = SVC()
svm.fit(vectors_train, train_labels)
print(np.mean(svm.predict(vectors_test) == test_labels))

In [ ]:
svm = SVC()
svm.fit(X_train[:nb_labelled, :], Y_train[:nb_labelled])
print(np.mean(svm.predict(X_test) == Y_test))

In [ ]:
# proportion = 0.8
nb_labelled = 1000 #int(proportion * X_train.shape[0])
for m in [2, 500, 1000, 2000, 2700]:
    s3vm.fit(X_train[:nb_labelled, :], Y_train[:nb_labelled], X_train[nb_labelled:nb_labelled+m, :], iter_gamma = 5, epochs = 3)
    print(s3vm.accuracy(X_test, Y_test))

In [ ]:
plt.plot(np.arange(len(s3vm.loss_list)), np.log10(s3vm.loss_list))
plt.show()